# Snowpark for ML - Diabetes Data Preparation Example

This notebook does the feature engineering for the diabetes model.

## 1. Get the data 

In [ ]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np

In [ ]:
# fetch dataset 
diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296) 
  
# data (as pandas dataframes) 
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features 
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets 

df = pd.concat([X,y], axis=1)
  
# variable information 
print(diabetes_130_us_hospitals_for_years_1999_2008.variables) 

## 2. Feature Engineering

In [ ]:
df['readmitted'] = df['readmitted'].replace({'NO': 0, '>30': 1, '<30': 1})
df['race'] = df['race'].replace({np.nan: 'Other'})
df = df[df['gender'] != 'Unknown/Invalid']

df['age'] = df['age'].replace({"[70-80)":"70+",
                               "[60-70)":"[50-70)",
                               "[50-60)":"[50-70)",
                               "[80-90)":"70+",
                               "[40-50)":"[20-50)",
                               "[30-40)":"[20-50)",
                               "[90-100)":"70+",
                               "[20-30)":"[20-50)"})


df['admission_type_id'] = df['admission_type_id'].replace({1.0:"Emergency",
                                                           2.0:"Emergency",
                                                           3.0:"Elective",
                                                           4.0:"New Born",
                                                           5.0:np.nan,
                                                           6.0:np.nan,
                                                           7.0:"Trauma Center",
                                                           8.0:np.nan})

df['discharge_disposition_id'] = df['discharge_disposition_id'].replace(
    {1:"Discharged to Home",
     6:"Discharged to Home",
     8:"Discharged to Home",
     13:"Discharged to Home",
     19:"Discharged to Home",
     18:np.nan, 25:np.nan, 26:np.nan,
     2:"Other", 3:"Other", 4:"Other",
     5:"Other", 7:"Other", 9:"Other",
     10:"Other", 11:"Other", 12:"Other",
     14:"Other", 15:"Other", 16:"Other",
     17:"Other", 20:"Other", 21:"Other",
     22:"Other", 23:"Other", 24:"Other",
     27:"Other", 28:"Other", 29:"Other", 30:"Other"}
) 

df['admission_source_id'] = df['admission_source_id'].replace(
    {1:"Referral", 2:"Referral", 3:"Referral", 4:"Transfer",
     5:"Transfer", 6:"Transfer", 7:"Emergency", 8:"Other",
     9:"Other", 10:"Transfer", 11:"Other", 12:"Other",
     13:"Other", 14:"Other", 15:np.nan, 17:np.nan, 
     18:"Transfer", 19:"Other", 20:np.nan, 21:np.nan,
     22:"Transfer", 23:"Other", 24: "Other", 25:"Transfer",
     26: "Transfer"}
)

df['medical_specialty'] = df['medical_specialty'].replace(
    {"Orthopedics-Reconstructive": "Orthopedics",
     "Surgeon": "Surgery-General",
     "Surgery-Cardiovascular": "Surgery-Cardiovascular/Thoracic",
     "Surgery-Thoracic": "Surgery-Cardiovascular/Thoracic",
     "Pediatrics-Endocrinology": "Pediatrics",
     "Pediatrics-CriticalCare": "Pediatrics",
     "Pediatrics-Pulmonology": "Pediatrics",
     "Radiologist": "Radiology",
     "Oncology": "Hematology/Oncology",
     "Hematology": "Hematology/Oncology",
     "Gynecology": "ObstetricsandGynecology",
     "Obstetrics": "ObstetricsandGynecology"
     }
)
df['medical_specialty'] = df['medical_specialty'].replace(
    {spec: "Other" for spec in df['medical_specialty'].value_counts().index.values[15:]}
)


def map_diagnosis(data, cols):
    for col in cols:
        data.loc[(data[col].str.contains("V")) | (data[col].str.contains("E")), col] = -1
        data[col] = data[col].astype(np.float16)

    for col in cols:
        data["temp_diag"] = np.nan
        data.loc[(data[col]>=390) & (data[col]<=459) | (data[col]==785), "temp_diag"] = "Circulatory"
        data.loc[(data[col]>=460) & (data[col]<=519) | (data[col]==786), "temp_diag"] = "Respiratory"
        data.loc[(data[col]>=520) & (data[col]<=579) | (data[col]==787), "temp_diag"] = "Digestive"
        data.loc[(data[col]>=680) & (data[col]<=709) | (data[col]==782), "temp_diag"] = "Skin"
        data.loc[(data[col]>=240) & (data[col]<250) | (data[col]>251) & (data[col]<=279), "temp_diag"] = "Non-diabetes endocrine/metabolic"
        data.loc[(data[col]>=250) & (data[col]<251), "temp_diag"] = "Diabetes"
        data.loc[(data[col]>=800) & (data[col]<=999), "temp_diag"] = "Injury"
        data.loc[(data[col]>=710) & (data[col]<=739), "temp_diag"] = "Musculoskeletal"
        data.loc[(data[col]>=580) & (data[col]<=629) | (data[col] == 788), "temp_diag"] = "Genitourinary"
        data.loc[(data[col]>=140) & (data[col]<=239), "temp_diag"] = "Neoplasms"
        data.loc[(data[col]>=290) & (data[col]<=319), "temp_diag"] = "Mental"
        data.loc[(data[col]>=1) & (data[col]<=139), "temp_diag"] = "Infectious"

        data["temp_diag"] = data["temp_diag"].fillna("Other")
        data[col] = data["temp_diag"]
        data = data.drop("temp_diag", axis=1)

    return data

df = map_diagnosis(df, ["diag_1","diag_2","diag_3"])


df['diabetesMed'] = df['diabetesMed'].replace({'Yes': 1, 'No': 0})

df['change'] = df['change'].replace({'Ch': 1, 'No': 0})

drop_columns = ['weight', 'payer_code']

In [ ]:
meds = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'glipizide', 'glyburide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'tolazamide', 'insulin',
       'glyburide-metformin']


drop_columns = drop_columns + meds

In [ ]:
X, y = df.drop(drop_columns + ['readmitted'], axis=1), df['readmitted']
df_clean = pd.concat([X, y], axis=1)

In [ ]:
##save a local copy if needed
df_clean = pd.read_csv('diabetes_clean.csv')